In [20]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
from shapely.geometry import shape, Point
from tqdm import tqdm

### Connecting to CA Geologic map to identify the rock type that each well sits on

[Data from this map](https://maps-cadoc.opendata.arcgis.com/maps/9eba56d981df4f839769ce9a2adc01f4/explore)

In [2]:
def identify_geology(lat, lon):
    identify_url = (
        "https://gis.conservation.ca.gov/server/rest/services/"
        "CGS/Geologic_Map_of_California/MapServer/identify"
    )
    
    params = {
        "f": "json",
        "geometry": f"{lon},{lat}",
        "geometryType": "esriGeometryPoint",
        "sr": "4326",
        "mapExtent": f"{lon-0.01},{lat-0.01},{lon+0.01},{lat+0.01}",
        "imageDisplay": "800,600,96",
        "tolerance": 1,
        "returnGeometry": "false",
        "layers": "all",
    }
    
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
        )
    }
    
    try:
        response = requests.get(identify_url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data.get("results", [])
    
    except Exception as e:
        print(f"Error querying service: {e}")
        return None

# trying it on a random point in CA to see what the response looks like
if __name__ == "__main__":
    lat = 34.0522
    lon = -118.2437
    
    results = identify_geology(lat, lon)
    if results:
        print("Number of results:", len(results))
        for r in results:
            print(r)


Number of results: 1
{'layerId': 12, 'layerName': 'Generalized Rock Types', 'displayFieldName': 'PTYPE', 'value': 'Q', 'attributes': {'OBJECTID': '13872', 'PTYPE': 'Q', 'GENERAL_LITHOLOGY': 'marine and nonmarine (continental) sedimentary rocks', 'AGE': 'Pleistocene-Holocene', 'DESCRIPTION': 'Alluvium, lake, playa, and terrace deposits; unconsolidated and semi-consolidated. Mostly nonmarine, but includes marine deposits near the coast.', 'SHAPE': 'Polygon', 'SHAPE.STArea()': '3942876694.487556', 'SHAPE.STLength()': '1860311.283161'}}


### Hooking back into CA DWR wells to map rock type to depths

Getting station IDs and performing geologic mapping top stage.

In [69]:
url = "https://data.cnra.ca.gov/api/3/action/datastore_search"
params = {
    "resource_id": "af157380-fb42-4abf-b72a-6f9f98868077",
    "limit": 100000,
    "offset": 0
}
response = requests.get(url, params=params)
data = response.json()['result']['records']
station_mapping = pd.DataFrame(data)
station_depth = station_mapping[~station_mapping.well_depth.isna()].copy()# approx 15k stations

In [4]:
def station_geologic(row):
    lat = row.latitude
    lon = row.longitude
    geology = identify_geology(lat, lon)
    rock_type = [item for item in geology if item["layerId"] == 12]
    if len(rock_type) > 1:
        print(f"Point has multiple geology: {len(rock_type)}")
    elif len(rock_type) == 0:
        return []
    try:
        return [rock["attributes"]["PTYPE"] for rock in rock_type]
    except KeyError:
        print(f"Warning: no geology found at {lat}, {lon} !")
        return None

In [5]:
'''
deprecated, method gets disconnected from geology server

rocks = []
for idx, row in tqdm(station_depth.iterrows(), total=len(station_depth)):
    val = station_geologic(row)
    rocks.append(val)
    '''

'\ndeprecated, method gets disconnected from geology server\n\nrocks = []\nfor idx, row in tqdm(station_depth.iterrows(), total=len(station_depth)):\n    val = station_geologic(row)\n    rocks.append(val)\n    '

In [52]:
def get_geologic_polygons(url):
    """
    Fetches the GeoJSON data from the given URL and returns a list of (shapely_geometry, properties).
    Each item in the returned list is a tuple of (Polygon/MultiPolygon, attribute_dictionary).
    """
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
        )
    }

    polygons = []
    offset = 0
    limit = 2000  # Adjust this based on the server's limit

    while True:
        print(f"Fetching data from: {url} with offset {offset}")
        paginated_url = f"{url}&resultRecordCount={limit}&resultOffset={offset}"
        response = requests.get(paginated_url, headers=headers)
        response.raise_for_status()
        
        geojson_data = response.json()
        features = geojson_data.get('features', [])
        
        if not features:
            break  # No more features to fetch
        
        for feature in features:
            geometry = shape(feature['geometry'])
            polygons.append((geometry, feature['properties']))
        
        offset += limit
    
    return polygons

def find_rock_type_for_points(points, polygons):
    """
    Given a list of (lon, lat) points and a list of (shapely_geometry, properties),
    returns a dict mapping each (lon, lat) point to a list of PTYPE values from the polygons it falls in.
    
    If a point does not fall in any polygon, it will map to an empty list.
    """
    results = {}
    
    for lon, lat in tqdm(points):
        point = Point(lon, lat)
        ptypes = []  # List to store all PTYPE values for the point
        
        # Check each polygon to see if the point is contained in it
        for polygon, props in polygons:
            if polygon.contains(point):
                ptype = props.get("PTYPE")  # Get the PTYPE value from the properties
                if ptype:
                    ptypes.append(ptype)
        
        # Ensure every point is included in the results, even if it has no PTYPE values
        results[(lon, lat)] = ptypes
    
    return results

url = (
        "https://gis.conservation.ca.gov/server/rest/services/CGS/"
        "Geologic_Map_of_California/MapServer/12/query"
        "?where=1=1&outFields=*&returnGeometry=true&outSR=4326&f=geojson"
    )
    
# Fetch polygons
polygons_data = get_geologic_polygons(url)
print(f"Total polygons fetched: {len(polygons_data)}")

Fetching data from: https://gis.conservation.ca.gov/server/rest/services/CGS/Geologic_Map_of_California/MapServer/12/query?where=1=1&outFields=*&returnGeometry=true&outSR=4326&f=geojson with offset 0
Fetching data from: https://gis.conservation.ca.gov/server/rest/services/CGS/Geologic_Map_of_California/MapServer/12/query?where=1=1&outFields=*&returnGeometry=true&outSR=4326&f=geojson with offset 2000
Fetching data from: https://gis.conservation.ca.gov/server/rest/services/CGS/Geologic_Map_of_California/MapServer/12/query?where=1=1&outFields=*&returnGeometry=true&outSR=4326&f=geojson with offset 4000
Fetching data from: https://gis.conservation.ca.gov/server/rest/services/CGS/Geologic_Map_of_California/MapServer/12/query?where=1=1&outFields=*&returnGeometry=true&outSR=4326&f=geojson with offset 6000
Fetching data from: https://gis.conservation.ca.gov/server/rest/services/CGS/Geologic_Map_of_California/MapServer/12/query?where=1=1&outFields=*&returnGeometry=true&outSR=4326&f=geojson with 

In [49]:
wells = [(i,j) for i,j in zip(station_depth.longitude, station_depth.latitude)]
geologies = find_rock_type_for_points(wells, polygons_data)

100%|██████████| 14563/14563 [16:32<00:00, 14.67it/s]


In [56]:
def find_row_geology(row):
    lat = row.latitude
    lon = row.longitude
    return geologies[(lon,lat)]

In [67]:
station_depth.apply(find_row_geology, axis=1).apply(lambda x: len(x) == 0).sum()

18

In [70]:
station_depth["geology"] = station_depth.apply(find_row_geology, axis=1)

In [72]:
station_depth.to_csv("stations_depth_geology.csv", index=False)